In [ ]:
# 1. Gerekli Kütüphaneler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.utils import class_weight
from PIL import Image

In [ ]:
# 2. Parametreler
img_size = (224, 224)
batch_size = 32
data_dir = 'cnndata/'  # Ana veri dizini
epochs = 15

In [ ]:
# 3. Veri Yükleme ve Ön İşleme
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_val_datagen = ImageDataGenerator(rescale=1. / 255)

train_data = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

val_data = test_val_datagen.flow_from_directory(
    os.path.join(data_dir, 'valid'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

test_data = test_val_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

In [ ]:
# 4. Sınıf Ağırlıkları
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
# 5. ResNet50 Modeli
base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(img_size[0], img_size[1], 3)
)
base_model.trainable = False  # İlk aşamada donuk katmanlar (transfer learning)

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

In [ ]:
# 6. Callback'ler
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'best_resnet50_model.h5',
    monitor='val_loss',
    save_best_only=True
)

In [ ]:
# 7. Model Derleme
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# 8. Model Eğitimi
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    class_weight=class_weights_dict,
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

In [ ]:
# 9. Eğitim Grafikleri
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title("Doğruluk")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Kayıp")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)

plt.tight_layout()
plt.savefig('training_plots.png')
plt.show()

In [ ]:
# 10. Performans Değerlendirme
loss, accuracy = model.evaluate(val_data, verbose=0)
print(f"\nDoğrulama Kaybı: {loss:.4f}")
print(f"Doğrulama Doğruluğu: {accuracy:.4f}")

val_pred = model.predict(val_data)
val_pred_classes = (val_pred > 0.5).astype(int).flatten()
val_true_classes = val_data.classes

print("\nSınıflandırma Raporu:")
print(classification_report(val_true_classes, val_pred_classes, target_names=list(train_data.class_indices.keys())))

In [ ]:
# 11. Modeli Kaydet
model.save('Web/models/final-resnet50.h5')